In [1]:
import pandas as pd

# Load input data

In [41]:
cod_post_original_df = pd.read_csv('input_data\codigos_postais.csv')
cod_post_mapping_df = pd.read_csv('input_data\Mapeamento_Freguesias_Pre_Pos_RATF.csv', skiprows=2, encoding = "LATIN-1")

In [30]:
cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,cod_arteria,tipo_arteria,prep1,titulo_arteria,prep2,nome_arteria,local_arteria,troco,porta,cliente,num_cod_postal,ext_cod_postal,desig_postal
0,1,1,249,Alcafaz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,11,AGADÃO
1,1,1,250,Caselho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,12,AGADÃO
2,1,1,251,Corga da Serra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,13,AGADÃO
3,1,1,252,Foz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,14,AGADÃO
4,1,1,253,Guistola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,15,AGADÃO


In [31]:
cod_post_mapping_df.head()

,Distrito/Ilha,Concelho,Freguesia Pré RATF,Alteração RATF,Freguesia Final (Pós RATF)
0,Aveiro,Águeda,Aguada de Cima,Sem alteração,Aguada de Cima
1,Aveiro,Águeda,Fermentelos,Sem alteração,Fermentelos
2,Aveiro,Águeda,Macinhata do Vouga,Sem alteração,Macinhata do Vouga
3,Aveiro,Águeda,Valongo do Vouga,Sem alteração,Valongo do Vouga
4,Aveiro,Águeda,Águeda,Agregação,União das freguesias de Águeda e Borralha


Check missing values:

In [102]:
cod_post_original_df.isna().sum()

cod_distrito            0
cod_concelho            0
cod_localidade          0
nome_localidade         0
cod_arteria         31213
tipo_arteria        33867
prep1              167514
titulo_arteria     281817
prep2              320537
nome_arteria        31213
local_arteria      253824
troco              286569
porta              322490
cliente            321222
num_cod_postal          0
ext_cod_postal          0
desig_postal            0
cod_postal              0
dtype: int64

We are not interested in any of the columns with missing values, so we can safely delete them:

In [108]:
cod_post_original_df.dropna(axis=1, inplace=True)
cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal,cod_postal
0,1,1,249,Alcafaz,3750,11,AGADÃO,3750-11
1,1,1,250,Caselho,3750,12,AGADÃO,3750-12
2,1,1,251,Corga da Serra,3750,13,AGADÃO,3750-13
3,1,1,252,Foz,3750,14,AGADÃO,3750-14
4,1,1,253,Guistola,3750,15,AGADÃO,3750-15


In [109]:
cod_post_mapping_df.isna()

Distrito/Ilha                 0
Concelho                      0
Freguesia Pré RATF            1
Alteração RATF                0
Freguesia Final (Pós RATF)    1
dtype: int64

In [64]:
cod_post_original_df['cod_postal'] = cod_post_original_df['num_cod_postal'].astype('str') + '-' + cod_post_original_df['ext_cod_postal'].astype('str')

cod_post_original_df[['num_cod_postal', 'ext_cod_postal', 'cod_postal']].head()

,num_cod_postal,ext_cod_postal,cod_postal
0,3750,11,3750-11
1,3750,12,3750-12
2,3750,13,3750-13
3,3750,14,3750-14
4,3750,15,3750-15
